In [ ]:
from rdkit import Chem
from rdkit.Chem.rdmolfiles import MolToXYZFile

from cct.rdkit.conformers import embed

mol = Chem.MolFromSmiles("CN1CCN(CC1)C2=NC3=C(C=C2)N(C(=O)N3C)CCC4=CC=CC=C4")
mol = Chem.AddHs(mol)
embed(mol)
charge = Chem.GetFormalCharge(mol)

MolToXYZFile(mol, "/tmp/bla/mol.xyz")
!cat /tmp/bla/mol.xyz

In [ ]:
from ase.io import read, write
from ase.optimize import BFGS
from tblite.ase import TBLite

atoms = read("/tmp/bla/mol.xyz")
atoms.calc = TBLite(method="GFN2-xTB", solvation=None)
opt = BFGS(atoms)
opt.run(fmax=0.05)
write("/tmp/bla/molopt.xyz", atoms)

In [ ]:
!cd /tmp/bla && crest mol.xyz --opt --chrg 0 --uhf 0 --alpb h2o --gfn 2
!cd /tmp/bla && crest molopt.xyz --gfn2 -chrg 0 --uhf 0 -T 14

In [ ]:
!cat /tmp/bla/crestopt.log

In [ ]:
_do_csearch(mol, "/tmp/bla")

In [ ]:
import os
import subprocess as sp

from rdkit import Chem
from rdkit.Chem.rdmolfiles import MolToXYZFile


def _do_csearch(
    mol,  # RDKit Mol object with conformer
    directory,
    gfn=2,
    nprocs=1,
    logfile=None,
    noncovalent=False,
    constraints=None,
    additional_flags=None,
    charge=0,
    multiplicity=1,
):
    assert isinstance(mol, Chem.Mol), "Input must be an RDKit Mol object."
    assert mol.GetNumConformers() > 0, "Mol must have at least one conformer."
    assert isinstance(directory, str)
    assert isinstance(nprocs, int)
    if logfile is not None:
        assert isinstance(logfile, str)
    assert gfn in [2, 1, "ff"], "Invalid value for `gfn`."

    os.makedirs(directory, exist_ok=True)
    xyz_path = os.path.join(directory, "xtb-in.xyz")
    MolToXYZFile(mol, xyz_path)

    nci = "-nci" if noncovalent else ""

    # Prepare crest command
    if constraints is not None:
        assert isinstance(constraints, list)
        assert all(isinstance(n, int) for n in constraints)
        # First create constrained input file with --constrain
        constrain_cmd = (
            f"crest xtb-in.xyz --constrain {','.join(str(c) for c in constraints)}"
        )
        result = sp.run(
            constrain_cmd, stdout=sp.PIPE, stderr=sp.PIPE, cwd=directory, shell=True
        )
        result.check_returncode()
        command = (
            f"crest xtb-in.xyz --gfn{gfn} --chrg {charge} -cinp .xcontrol.sample "
            f"--uhf {multiplicity - 1} -T {nprocs} {nci}"
        )
    else:
        command = (
            f"crest xtb-in.xyz --gfn{gfn} --chrg {charge} "
            f"--uhf {multiplicity - 1} -T {nprocs} {nci}"
        )

    if additional_flags:
        command += f" {additional_flags}"

    # Run CREST
    if logfile:
        with open(logfile, "w") as f:
            result = sp.run(command, stdout=f, stderr=f, cwd=directory, shell=True)
    else:
        result = sp.run(
            command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=directory, shell=True
        )
    result.check_returncode()

    # Read ensemble
    ensemble_path = os.path.join(directory, "crest_conformers.xyz")
    if not os.path.isfile(ensemble_path):
        raise FileNotFoundError(f"{ensemble_path} not found after CREST run.")

    return ensemble_path  # or parse it into RDKit mols or structures if needed